In [5]:
print("hello world")

hello world


In [6]:
%pwd

'c:\\Users\\KRISHNA\\OneDrive\\Desktop\\SOC\\Generative-AI-for-medical-reports---SOC'

In [7]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
def load_data(data):
    """Load data from a directory."""
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_data("Data/")

KeyboardInterrupt: 

In [ ]:
extracted_data

NameError: name 'extracted_data' is not defined

In [17]:
def make_chunks(extracted_data):
    """Split documents into chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    chunks = text_splitter.split_documents(extracted_data)
    return chunks

In [18]:
text_chunks = make_chunks(extracted_data)
len(text_chunks)

NameError: name 'extracted_data' is not defined

In [ ]:
# text_chunks

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\KRISHNA\AppData\Local\Temp\ipykernel_16440\1185666148.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\KRISHNA\anaconda3\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
query_results = embedding_model.embed_query("What is the treatment for diabetes?")
print(len(query_results))

384


In [22]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [8]:
#create database index

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


index_name = "healtcare-report-bot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
        deletion_protection="disabled",
        tags={
            "environment": "development"
        }
    )

c:\Users\KRISHNA\anaconda3\envs\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at db_data_2025-04.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [26]:
#embedding all chunks and upserting data into pinecone index

from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    index_name=index_name,
    embedding=embedding_model,
    documents=text_chunks,
)

NameError: name 'text_chunks' is not defined

In [9]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model,
)

In [20]:
vector_store

In [11]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [15]:
retrieved_docs = retriever.invoke("What is the treatment for diabetes?")

In [23]:
print(retrieved_docs)

[Document(id='1e4356e5-25f1-45eb-9568-9f1fece3325d', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 434.0, 'page_label': '435', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 1047\nDiabetes mellitus'), Document(id='be2480ff-3343-4607-b7ed-4add9368670a', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 438.0, 'page_label': '439', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='available to treat diabetes include metformin, acarbose,\nand troglitizone. The choice of the right medication\ndepends in part on the individual patient 

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    api_key=GEMINI_API_KEY,
    model ="gemini-2.5-flash",
    temperature=0.4,
    max_tokens=500
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = """
You are a helpful medical assistant that generates structured medical reports.

Always follow these rules:
- Use the provided context and user query to generate a report.
- If the answer is not available in the context, say: "I'm not sure based on the available information."
- Avoid giving advice that only a doctor should provide.
- Output the answer in this exact format:

==== Personalized Healthcare Report ====

**Extracted Symptoms**: <comma-separated list of symptoms extracted from the user query>

**Disease**: <Likely disease name from context, or say "I'm not sure based on the available information.">

**Description**: <Short 2–4 sentence summary of the disease from context>

**Medicines**: <General medicines from context or "Consult doctor">

**Prevention/Precautions**:
- <Tip 1>
- <Tip 2>
- <Tip 3>
(Include 3–5 relevant items if possible)

**Matched Symptoms**: <number matched> / <total known symptoms for this disease>

========================================

Context:
{context}


"""



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [41]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [44]:
response = rag_chain.invoke({"input": "I have diabetes"})

In [45]:
print(response['answer'])

==== Medical Report ====
**Symptoms Mentioned**:
- None mentioned by user

**Likely Conditions / Diagnosis**:
- Diabetes Mellitus (as stated by user)

**Recommended Next Steps**:
- Continue to consult with a healthcare professional for ongoing management of diabetes.
- Discuss any new or worsening symptoms, or concerns about your condition with your doctor.
- Treatment for diabetes may include changes in diet, oral medications, and in some cases, daily injections of insulin.
- It is important to manage diabetes to prevent serious health complications such as renal (kidney) failure, heart disease, stroke, and blindness.
